In [ ]:
import scripts
import numpy as np
import tensorflow as tf
import importlib

In [ ]:
importlib.reload(scripts)

# Solving Chinese MNIST using Transfer Learning

## Some Hyperparameters

In [ ]:
NUMBER_OF_CLASSES = 15
IMAGE_SIZE = (64, 64)
IMAGE_SHAPE = IMAGE_SIZE + (3,)

CLASSIFIER_LEARNING_RATE = 0.0001
CLASSFIER_NUMBER_OF_EPOCHS = 10
BATCH_SIZE = 32
OPTIMIZER = tf.keras.optimizers.Adam
LOSS_MEASURE = tf.keras.losses.SparseCategoricalCrossentropy

## Dataloaders

In [ ]:
small_training_dataloader = tf.keras.utils.image_dataset_from_directory("./small_training_dataset/",
                                                                        batch_size=BATCH_SIZE,
                                                                        image_size=IMAGE_SIZE,
                                                                        seed=413)
medium_training_dataloader = tf.keras.utils.image_dataset_from_directory("./medium_training_dataset/",
                                                                         batch_size=BATCH_SIZE,
                                                                         image_size=IMAGE_SIZE,
                                                                         seed=413)
large_training_dataloader = tf.keras.utils.image_dataset_from_directory("./large_training_dataset/",
                                                                        batch_size=BATCH_SIZE,
                                                                        image_size=IMAGE_SIZE,
                                                                        seed=413)
validation_dataloader = tf.keras.utils.image_dataset_from_directory("./validation_dataset/",
                                                                    batch_size=BATCH_SIZE,
                                                                    image_size=IMAGE_SIZE,
                                                                    seed=413)

## Take a Peek

In [ ]:
scripts.peek_into_dataloader(small_training_dataloader)

In [ ]:
small_training_dataloader = small_training_dataloader.cache().prefetch(buffer_size=tf.data.AUTOTUNE)
medium_training_dataloader = medium_training_dataloader.cache().prefetch(buffer_size=tf.data.AUTOTUNE)
large_training_dataloader = large_training_dataloader.cache().prefetch(buffer_size=tf.data.AUTOTUNE)
validation_dataloader = validation_dataloader.cache().prefetch(buffer_size=tf.data.AUTOTUNE)

### Base Models

In [ ]:
small_efficientnet_base_model = tf.keras.applications.EfficientNetV2S(include_top=False, input_shape=IMAGE_SHAPE) # expects an input to be in the range [0, 255].
medium_efficientnet_base_model = tf.keras.applications.EfficientNetV2M(include_top=False, input_shape=IMAGE_SHAPE)
large_efficientnet_base_model = tf.keras.applications.EfficientNetV2L(include_top=False, input_shape=IMAGE_SHAPE)

### Full Models

In [ ]:
small_efficientnet = scripts.add_classifier(small_efficientnet_base_model, IMAGE_SHAPE, NUMBER_OF_CLASSES)
medium_efficientnet = scripts.add_classifier(medium_efficientnet_base_model, IMAGE_SHAPE, NUMBER_OF_CLASSES)
large_efficientnet = scripts.add_classifier(large_efficientnet_base_model, IMAGE_SHAPE, NUMBER_OF_CLASSES)

### Inspection

In [ ]:
print("Number of layers in the small efficientnet base model: {}".format(len(small_efficientnet.layers[1].layers)))
print("Number of layers in the medium efficientnet base model: {}".format(len(medium_efficientnet.layers[1].layers)))
print("Number of layers in the large efficientnet base model: {}".format(len(large_efficientnet.layers[1].layers)))

In [ ]:
small_efficientnet.summary()

In [ ]:
medium_efficientnet.summary()

In [ ]:
large_efficientnet.summary()

## Training

In [ ]:
CHECKPOINT_NAMES = np.array([["small_efficientnet_with_small_dataset", "small_efficientnet_with_medium_dataset", "small_efficientnet_with_large_dataset"],
                             ["medium_efficientnet_with_small_dataset", "medium_efficientnet_with_medium_dataset", "medium_efficientnet_with_large_dataset"],
                             ["large_efficientnet_with_small_dataset", "large_efficientnet_with_medium_dataset", "large_efficientnet_with_large_dataset"]])
MODELS = (small_efficientnet, medium_efficientnet, large_efficientnet)
DATALOADERS = (small_training_dataloader, medium_training_dataloader, large_training_dataloader)

## Training a Classifier

In [ ]:
accuracies_after_training_classifiers = scripts.train_classifiers(MODELS,
                                                                  DATALOADERS,
                                                                  validation_dataloader,
                                                                  OPTIMIZER(learning_rate=CLASSIFIER_LEARNING_RATE),
                                                                  LOSS_MEASURE(from_logits=True),
                                                                  CLASSFIER_NUMBER_OF_EPOCHS,
                                                                  CHECKPOINT_NAMES)

## Fine-Tuning

Only fine tune after the classifer have been trained.

### Some Hyperparameters

In [ ]:
FINE_TUNING_LEARNING_RATE = CLASSIFIER_LEARNING_RATE / 10
FINE_TUNING_NUMBER_OF_EPOCHS = 10
TOTAL_NUMBER_OF_EPOCHS = CLASSFIER_NUMBER_OF_EPOCHS + FINE_TUNING_NUMBER_OF_EPOCHS
PERCENTAGE_OF_LAYERS_TO_FREEZE = [0.2, 0.4, 0.6]

In [ ]:
scripts.fine_tune(MODELS, DATALOADERS, validation_dataloader, OPTIMIZER(learning_rate=CLASSIFIER_LEARNING_RATE), LOSS_MEASURE(from_logits=True), TOTAL_NUMBER_OF_EPOCHS, CHECKPOINT_NAMES, PERCENTAGE_OF_LAYERS_TO_FREEZE)

## Results